<h1><center> PPOL 5203 Data Science I: Foundations <br><br> 
<font color='grey'> Tidy Data and Joining Methods in Pandas<br><br>
Tiago Ventura</center> <h1> 

---

**In this Notebook we cover**

This is our last notebook of data wrangling with `Pandas`. We will manly cover: 

- Tidy Data
- Joining Methods in Pandas


In [2]:
import pandas as pd
import numpy as np

## Tidy Data

Data can be organized in many different ways and can target many different concepts. Having an consistent and well-established procedures for data organization will make your workflow will faster, more efficient and less prone to errors. 

On tasks related to data visualization and data wrangling, we will often try to organize our datasets following a `tidy data format` proposed by [Hadley Wickham in his 2014 article](https://vita.had.co.nz/papers/tidy-data.pdf). 

Consider the following 4 ways to organize the same data (_example pulled from [R4DS](https://r4ds.had.co.nz/tidy-data.html)_).

**<center> Example 1 </center>**

| Country     |Year| Cases | Population|
|:-----------:|:--:|:-----:|:---------:|
|Afghanistan  |1999|    745|   19987071|
|Afghanistan  |2000|   2666|   20595360|
|Brazil       |1999|  37737|  172006362|
|Brazil       |2000|  80488|  174504898|
|China        |1999| 212258| 1272915272|
|China        |2000| 213766| 1280428583|

**<center> Example 2 </center>**

|country      |year |type      |     count|
|:-----------:|:--:|:-----:|:---------:|
|Afghanistan  |1999 |cases     |       745|
|Afghanistan  |1999 |population|  19987071|
|Afghanistan  |2000 |cases     |      2666|
|Afghanistan  |2000 |population|  20595360|
|Brazil       |1999 |cases     |     37737|
|Brazil       |1999 |population| 172006362|

**<center> Example 3 </center>**

|country      |year| rate             |
|:-----------:|:--:|:-----:|
|Afghanistan  |1999| 745/19987071     |
|Afghanistan  |2000| 2666/20595360    |
|Brazil       |1999| 37737/172006362  |
|Brazil       |2000| 80488/174504898  |
|China        |1999| 212258/1272915272|
|China        |2000| 213766/1280428583|

**<center> Example 4 </center>**

| country     |`1999` |`2000`|
|:-----------:|:--:|:-----:|
| Afghanistan |   745 |  2666|
| Brazil      | 37737 | 80488|
| China       |212258 |213766|
    
    
    
| country     |    `1999`|     `2000`|
|:-----------:|:--:|:-----:|
| Afghanistan |  19987071|   20595360|
| Brazil      | 172006362|  174504898|
| China       |1272915272| 1280428583|

<div class="alert alert-block alert-info">

**Of the data examples outlined above, only the first could be considered `tidy` by this definition.**
</div>

### What makes a data tidy?

Three interrelated rules which make a dataset **tidy**:
<br><br>

1. **Each variable must have its own column.**


2. **Each observation must have its own row.**


3. **Each value must have its own cell.**

![Image drawn from Grolemund and Wickham 2017](https://d33wubrfki0l68.cloudfront.net/6f1ddb544fc5c69a2478e444ab8112fb0eea23f8/91adc/images/tidy-1.png)


### Why Tidy?

There are many reasons why `tidy format` facilitates data analysis:  

- Facilitates split-apply-combine analysis
- Take full advantage of `pandas` vectorize operations over columns,
- Allows for apply operations over unit of your data 
- Sits well with grammar of graphs approach for visualization

<div class="alert alert-block alert-info">

You don't need to be convinced in theory of these advantages. You will get in practice the tasty of working with tidy format data **The most important thing for you to have now is to follow a consistent strategy to organize your datasets, and apply these procedures across the board!**
    
</div>    

## Using `pandas` to tidy your data. 

Most often you will encounter untidy datasets. A huge portion of your time as a data scientist will consist on apply tidy procedures to your dataset before starting any analysis or modeling. Let's learn some `pandas` methods for it!

Let's first create every example of datasets we saw above

In [3]:
base_url = "https://github.com/byuidatascience/data4python4ds/raw/master/data-raw/"
table1 = pd.read_csv("{}table1/table1.csv".format(base_url))
table2 = pd.read_csv("{}table2/table2.csv".format(base_url))
table3 = pd.read_csv("{}table3/table3.csv".format(base_url))
table4a = pd.read_csv("{}table4a/table4a.csv".format(base_url), names=["country", "cases_1999", "cases_2000"] )
table4b = pd.read_csv("{}table4b/table4b.csv".format(base_url), names=["country", "population_1999", "poulation_2000"] )
table5 = pd.read_csv("{}table5/table5.csv".format(base_url), dtype = 'object')

In [4]:
# see
table1

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


For most real analyses, you will need to resolve one of three common problems to tidy your data:

- One variable might be spread across multiple columns (`pd.melt()`)

- One observation might be scattered across multiple rows (`pd.pivot_table()`)

- A cell might contain weird/non-sensical/missing values. 

We will focus on the first two cases, as the last requires a mix of data cleaning skills that are spread over different notebooks

#### `pd.melt()` from wide to long


![https://r4ds.had.co.nz/tidy-data.html](https://d33wubrfki0l68.cloudfront.net/3aea19108d39606bbe49981acda07696c0c7fcd8/2de65/images/tidy-9.png)

Requires: 

- Columns whose names are identifier variables, and you wish to keep in the dataset as it is. 

- A string for the new column with the variable names. 

- A string for the new column with the values.


In [5]:
# untidy - wide
print(table4a)

       country  cases_1999  cases_2000
0      country        1999        2000
1  Afghanistan         745        2666
2       Brazil       37737       80488
3        China      212258      213766


In [6]:
# tidy - from wide to long
table4a.melt(id_vars=['country'], var_name = "year", value_name = "cases")

,country,year,cases
0,country,cases_1999,1999
1,Afghanistan,cases_1999,745
2,Brazil,cases_1999,37737
3,China,cases_1999,212258
4,country,cases_2000,2000
5,Afghanistan,cases_2000,2666
6,Brazil,cases_2000,80488
7,China,cases_2000,213766


In [7]:
help(pd.melt)

Help on function melt in module pandas.core.reshape.melt:

melt(frame: 'DataFrame', id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index: 'bool' = True) -> 'DataFrame'
    Unpivot a DataFrame from wide to long format, optionally leaving identifiers set.
    
    This function is useful to massage a DataFrame into a format where one
    or more columns are identifier variables (`id_vars`), while all other
    columns, considered measured variables (`value_vars`), are "unpivoted" to
    the row axis, leaving just two non-identifier columns, 'variable' and
    'value'.
    
    Parameters
    ----------
    id_vars : tuple, list, or ndarray, optional
        Column(s) to use as identifier variables.
    value_vars : tuple, list, or ndarray, optional
        Column(s) to unpivot. If not specified, uses all columns that
        are not set as `id_vars`.
    var_name : scalar
        Name to use for the 'variable' column. If None it uses
        ``fr

#### `pd.pivot_table()` from long to wide (but tidy)

![](https://d33wubrfki0l68.cloudfront.net/8350f0dda414629b9d6c354f87acf5c5f722be43/bcb84/images/tidy-8.png)

`pivot_table()` is the opposite of melt(). Think about this as you are widening a coarced variables. It requires: 

- Index to hold your new dataset upon

- The column to open up across multiple new columnes. 

- The column with the values to fill the cell on the new colum. 

In [121]:
#untidy
print(table2)

#tidy
table2_tidy = table2.pivot_table(
    index = ['country', 'year'], 
    columns = 'type', 
    values = 'count')

        country  year        type       count
0   Afghanistan  1999       cases         745
1   Afghanistan  1999  population    19987071
2   Afghanistan  2000       cases        2666
3   Afghanistan  2000  population    20595360
4        Brazil  1999       cases       37737
5        Brazil  1999  population   172006362
6        Brazil  2000       cases       80488
7        Brazil  2000  population   174504898
8         China  1999       cases      212258
9         China  1999  population  1272915272
10        China  2000       cases      213766
11        China  2000  population  1280428583


In [122]:
table2_tidy

type               cases  population
country     year                    
Afghanistan 1999     745    19987071
            2000    2666    20595360
Brazil      1999   37737   172006362
            2000   80488   174504898
China       1999  212258  1272915272
            2000  213766  1280428583

## string methods with `pandas`

To tidy example three, we will use a combination of `pandas` and `string` methods. This combination will be around very often in our data cleaning tasks

In [123]:
# we need to separate rate
table3

,country,year,rate
0,Afghanistan,1999,745/19987071
1,Afghanistan,2000,2666/20595360
2,Brazil,1999,37737/172006362
3,Brazil,2000,80488/174504898
4,China,1999,212258/1272915272
5,China,2000,213766/1280428583


#### `str.split()`: to split strings in multiple elements

In [8]:
# simple string
hello, world = "hello/world".split("/")
print(hello)
print(world)

# with pandas
table3[["cases","population"]]=table3['rate'].str.split("/",  expand=True)


hello
world


In [9]:
table3

,country,year,rate,cases,population
0,Afghanistan,1999,745/19987071,745,19987071
1,Afghanistan,2000,2666/20595360,2666,20595360
2,Brazil,1999,37737/172006362,37737,172006362
3,Brazil,2000,80488/174504898,80488,174504898
4,China,1999,212258/1272915272,212258,1272915272
5,China,2000,213766/1280428583,213766,1280428583


`string` methods will be super helpful on cleaning your data

In [10]:
# all to upper
table3["country_upper"]=table3["country"].str.upper()

# length
table3["country_length"]=table3["country"].str.len()

# find 
table3["country_brazil"]=table3["country"].str.find("Brazil")


# replace
table3["country_brazil"]=table3["country"].str.replace("Brazil", "BR")

# extract
table3[["century", "years"]] = table3["year"].astype(str).str.extract("(\d{2})(\d{2})")

# see all
table3

,country,year,rate,cases,population,country_upper,country_length,country_brazil,century,years
0,Afghanistan,1999,745/19987071,745,19987071,AFGHANISTAN,11,Afghanistan,19,99
1,Afghanistan,2000,2666/20595360,2666,20595360,AFGHANISTAN,11,Afghanistan,20,00
2,Brazil,1999,37737/172006362,37737,172006362,BRAZIL,6,BR,19,99
3,Brazil,2000,80488/174504898,80488,174504898,BRAZIL,6,BR,20,00
4,China,1999,212258/1272915272,212258,1272915272,CHINA,5,China,19,99
5,China,2000,213766/1280428583,213766,1280428583,CHINA,5,China,20,00


## Joining Methods

It is unlikely that your work as a data scientist will be restricted to analyze one isolated data frame -- or table in the `SQL`/Database lingo. Most often you have multiple tables of data, and your work will consist of combining them to answer the questions that you’re interested in. 

There two major reasons for why complex datasets are often stored across multiple tables: 

- A) Integrity and efficiency issues often referred as [database normalization](https://en.wikipedia.org/wiki/Database_normalization). As your data grow in size and complexity, keeping a unified database leads to redundancy and possible errors on data entry. 

- B) Data comes from different sources. As a researcher, you are being creative and augmenting the information at your hand to answer a policy question. 

Database normalization works as an <span style='color:blue'> **constraint**</span>, a guardrail to protect your data infrastructure. The second reason for why joining methods matter is primarily an <span style='color:red'> **opportunity** </span>. Keep always your eyes open for creative ways to connect data sources. Very critical research ideas might emerge from data augmentation from joining initially conceived unrelated datasets.

### `pandas` methods:

`pandas` comes baked in with a fully functional method (`pd.merge`) to join data. However, we'll use the `SQL` language when talking about joins to stay consistent with `SQL` and `R Tidyverse`. 

Let's start creating two tables for us to play around with `pandas` join methods

In [12]:
# Two fake data frames
import pandas as pd
data_x = pd.DataFrame(dict(key = ["1","2","3"],
                           var_x = ["x1","x2","x3"]))
data_y = pd.DataFrame(dict(key = ["1","2","4"],
                           var_y = ["y1","y2","y4"]))
display(data_x)
display(data_y)

,key,var_x
0,1,x1
1,2,x2
2,3,x3


,key,var_y
0,1,y1
1,2,y2
2,4,y4


### Left Join: `pd.merge(<data>, how="left")`

- Keep all keys from the data set of the right

<br><br>

<div>
<img src="./figs/left_join.png" width="500"/>
</div>


In [13]:
# chaining datasets
data_x.merge(data_y,how="left") 

,key,var_x,var_y
0,1,x1,y1
1,2,x2,y2
2,3,x3,NaN


In [14]:
# calling the construct
pd.merge(data_x, data_y, how="left")

,key,var_x,var_y
0,1,x1,y1
1,2,x2,y2
2,3,x3,NaN


### Right Join:  `pd.merge(<data>, how="right")`

- Keep all keys from the data set of the right

<br><br>

<div>
<img src="./figs/right_join.png" width="500"/>
</div>

In [15]:
# chaining datasets
data_x.merge(data_y,how="right") 

,key,var_x,var_y
0,1,x1,y1
1,2,x2,y2
2,4,NaN,y4


### Full (outer) Join: `pd.merge(<data>, how="outer")`

- Keep all keys from left and right

<br><br>


<div>
<img src="./figs/full_join.png" width="500"/>
</div>

In [16]:
# chaining datasets
data_x.merge(data_y,how="outer") 

,key,var_x,var_y
0,1,x1,y1
1,2,x2,y2
2,3,x3,NaN
3,4,NaN,y4


### Inner Join

- Keep only matched keys

<br><br>

![](https://d33wubrfki0l68.cloudfront.net/3abea0b730526c3f053a3838953c35a0ccbe8980/7f29b/diagrams/join-inner.png)

In [17]:
# chaining datasets
data_x.merge(data_y,how="inner") 

,key,var_x,var_y
0,1,x1,y1
1,2,x2,y2


### Handling disparate column names

In [18]:
# rename datasets
data_X = data_x.rename(columns={"key":"country_x"})
data_Y = data_y.rename(columns={"key":"country_y"})

# join now, and you will get an error
pd.merge(data_X,
         data_Y,
         how="left",
         left_on = "country_x",  # The left column naming convention 
         right_on="country_y") # The right column naming convention )

,country_x,var_x,country_y,var_y
0,1,x1,1,y1
1,2,x2,2,y2
2,3,x3,NaN,NaN


In [19]:
help(pd.merge)

Help on function merge in module pandas.core.reshape.merge:

merge(left: 'DataFrame | Series', right: 'DataFrame | Series', how: 'str' = 'inner', on: 'IndexLabel | None' = None, left_on: 'IndexLabel | None' = None, right_on: 'IndexLabel | None' = None, left_index: 'bool' = False, right_index: 'bool' = False, sort: 'bool' = False, suffixes: 'Suffixes' = ('_x', '_y'), copy: 'bool' = True, indicator: 'bool' = False, validate: 'str | None' = None) -> 'DataFrame'
    Merge DataFrame or named Series objects with a database-style join.
    
    A named Series object is treated as a DataFrame with a single named column.
    
    The join is done on columns or indexes. If joining columns on
    columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
    on indexes or indexes on a column or columns, the index will be passed on.
    When performing a cross merge, no column specifications to merge on are
    allowed.
    
    .. warning::
    
        If both key columns con

### Concatenating by columns and rows


#### By Rows: `pd.concat(<>, axis=0)`
<br><br>
![](./figs/rbind.png)

In [20]:
# full of NAS because the second columnes do not have the same name
pd.concat([data_x,data_y],
            sort=False) # keep the original structure

,key,var_x,var_y
0,1,x1,NaN
1,2,x2,NaN
2,3,x3,NaN
0,1,NaN,y1
1,2,NaN,y2
2,4,NaN,y4


#### By Columns: `pd.concat(<>, axis=1)`

<br><br>
![](./figs/cbind.png)

In [21]:
pd.concat([data_x,data_y],axis=1)

,key,var_x,key,var_y
0,1,x1,1,y1
1,2,x2,2,y2
2,3,x3,4,y4


Note that when we row bind two `DataFrame` objects, `pandas` will preserve the indices. And you can use this to sort your dataset. 

In [22]:
pd.concat([data_x,data_y],axis=0).sort_index()

,key,var_x,var_y
0,1,x1,NaN
0,1,NaN,y1
1,2,x2,NaN
1,2,NaN,y2
2,3,x3,NaN
2,4,NaN,y4


To keep the data tidy, we can preserve which data is coming from where by generating a hierarchical index using the `key` argument. Of course, this could also be done by creating a unique columns in each dataset before the join.

In [23]:
pd.concat([data_x,data_y],axis=0, keys=["data_x","data_y"])

key var_x var_y
data_x 0   1    x1   NaN
       1   2    x2   NaN
       2   3    x3   NaN
data_y 0   1   NaN    y1
       1   2   NaN    y2
       2   4   NaN    y4

Lastly, note that we can completely ignore the index if need be.

In [24]:
pd.concat([data_x,data_y],axis=0,ignore_index=True)

,key,var_x,var_y
0,1,x1,NaN
1,2,x2,NaN
2,3,x3,NaN
3,1,NaN,y1
4,2,NaN,y2
5,4,NaN,y4


## Practice

Using both the World Cup Datasets from the Data Wrangling Notebook:
    
1. Select the following columns: MatchID, Year, Stage, Home Team Name, Away Team Name, Home Team Goals, Away Team Goals.
    
2. Clean the column names converting all to lower case and removing spaces.    
    
3. Convert this data to the long format, having: a) a single column for the teams playing; b) a single column for the goals scored.  

4. Which team have played more games in the history of the world cup?

5. Which team have the higher average of goals in the history of the world cup?
    

In [25]:
# open datasets
wc = pd.read_csv("WorldCups.csv")
wc_matches = pd.read_csv("WorldCupMatches.csv")
